<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_fgsm_MLP_SHAP_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 9.7 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load trainset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/multi/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385156 entries, 0 to 385155
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      385156 non-null  float64
 1   Tot Fwd Pkts       385156 non-null  float64
 2   Tot Bwd Pkts       385156 non-null  float64
 3   TotLen Fwd Pkts    385156 non-null  float64
 4   TotLen Bwd Pkts    385156 non-null  float64
 5   Fwd Pkt Len Max    385156 non-null  float64
 6   Fwd Pkt Len Min    385156 non-null  float64
 7   Fwd Pkt Len Mean   385156 non-null  float64
 8   Fwd Pkt Len Std    385156 non-null  float64
 9   Bwd Pkt Len Max    385156 non-null  float64
 10  Bwd Pkt Len Min    385156 non-null  float64
 11  Bwd Pkt Len Mean   385156 non-null  float64
 12  Bwd Pkt Len Std    385156 non-null  float64
 13  Flow Byts/s        385156 non-null  float64
 14  Flow Pkts/s        385156 non-null  float64
 15  Flow IAT Mean      385156 non-null  float64
 16  Fl

In [4]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385156 entries, 0 to 385155
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      385156 non-null  float64
 1   Tot Fwd Pkts       385156 non-null  float64
 2   Tot Bwd Pkts       385156 non-null  float64
 3   TotLen Fwd Pkts    385156 non-null  float64
 4   TotLen Bwd Pkts    385156 non-null  float64
 5   Fwd Pkt Len Max    385156 non-null  float64
 6   Fwd Pkt Len Min    385156 non-null  float64
 7   Fwd Pkt Len Mean   385156 non-null  float64
 8   Fwd Pkt Len Std    385156 non-null  float64
 9   Bwd Pkt Len Max    385156 non-null  float64
 10  Bwd Pkt Len Min    385156 non-null  float64
 11  Bwd Pkt Len Mean   385156 non-null  float64
 12  Bwd Pkt Len Std    385156 non-null  float64
 13  Flow Byts/s        385156 non-null  float64
 14  Flow Pkts/s        385156 non-null  float64
 15  Flow IAT Mean      385156 non-null  float64
 16  Fl

# Load adv samples

In [5]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [6]:
fgsm_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/fgsm/adv_fgsm_multi_target.csv'
#fgsm_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/fgsm/adv_fgsm_target.csv'

In [7]:
fgsm_df = pd.read_csv(fgsm_path)

In [8]:
fgsm_df.columns = feature_list
fgsm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77203 entries, 0 to 77202
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      77203 non-null  float64
 1   Tot Fwd Pkts       77203 non-null  float64
 2   Tot Bwd Pkts       77203 non-null  float64
 3   TotLen Fwd Pkts    77203 non-null  float64
 4   TotLen Bwd Pkts    77203 non-null  float64
 5   Fwd Pkt Len Max    77203 non-null  float64
 6   Fwd Pkt Len Min    77203 non-null  float64
 7   Fwd Pkt Len Mean   77203 non-null  float64
 8   Fwd Pkt Len Std    77203 non-null  float64
 9   Bwd Pkt Len Max    77203 non-null  float64
 10  Bwd Pkt Len Min    77203 non-null  float64
 11  Bwd Pkt Len Mean   77203 non-null  float64
 12  Bwd Pkt Len Std    77203 non-null  float64
 13  Flow Byts/s        77203 non-null  float64
 14  Flow Pkts/s        77203 non-null  float64
 15  Flow IAT Mean      77203 non-null  float64
 16  Flow IAT Std       772

In [ ]:
#fgsm_label = pd.read_csv(fgsm_label_path)
#fgsm_label.info()

# Load DL-based IDS model

In [9]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_mlp_multi.h5', compile=True)

# SHAP

In [10]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [11]:
fgsm_1000 = fgsm_df.head(1000)

In [12]:
shap_values = explainer.shap_values(fgsm_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
print(shap_values[0])

[[ 0.02048    -0.02055665  0.02652105 ...  0.00609531  0.00624188
   0.009305  ]
 [-0.00476057  0.         -0.01392147 ... -0.02094732  0.02225635
  -0.00413825]
 [ 0.02151127 -0.03236551  0.03678761 ... -0.02033149 -0.01213962
   0.        ]
 ...
 [-0.01058606  0.01020821  0.0251229  ...  0.          0.02635153
   0.0098412 ]
 [ 0.          0.          0.02608885 ...  0.         -0.02315402
   0.        ]
 [ 0.00451426  0.         -0.02923004 ...  0.          0.
  -0.0093804 ]]


In [14]:
type(shap_values[0])

numpy.ndarray

In [15]:
data_df = pd.DataFrame(shap_values[0])

In [16]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.020480,-0.020557,0.026521,0.000000,0.000000,0.096942,0.040675,0.077318,0.116713,0.014274,...,-0.049393,-0.028038,-0.010658,-0.033797,0.007396,0.000000,0.017359,0.006095,0.006242,0.009305
1,-0.004761,0.000000,-0.013921,0.000000,-0.041399,-0.010971,0.124678,-0.007265,0.273530,-0.088020,...,0.000000,0.000000,0.008414,0.000000,0.000000,-0.008715,0.039743,-0.020947,0.022256,-0.004138
2,0.021511,-0.032366,0.036788,0.000000,0.000000,0.124058,0.037773,0.053057,0.093899,0.015353,...,-0.058623,0.000000,-0.033571,0.000000,0.011193,0.004507,0.020120,-0.020331,-0.012140,0.000000
3,0.007138,0.000000,0.000000,-0.004327,-0.003638,-0.001787,0.000000,0.000000,0.000000,0.000000,...,0.005447,0.000000,-0.031665,0.000000,0.000000,-0.023587,0.004569,0.000000,-0.007034,0.000000
4,0.019640,0.000000,0.022095,0.031493,0.000000,0.071883,0.038417,0.078748,0.096731,0.017938,...,0.000000,-0.006971,-0.044562,-0.020881,0.000000,0.000000,0.016778,-0.009856,0.000000,0.000000


In [17]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_fgsm/1000_samples_[0].csv', index=False)

In [18]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.020480,-0.020557,0.026521,0.000000,0.000000,0.096942,0.040675,0.077318,0.116713,0.014274,...,-0.049393,-0.028038,-0.010658,-0.033797,0.007396,0.000000,0.017359,0.006095,0.006242,0.009305
1,-0.004761,0.000000,-0.013921,0.000000,-0.041399,-0.010971,0.124678,-0.007265,0.273530,-0.088020,...,0.000000,0.000000,0.008414,0.000000,0.000000,-0.008715,0.039743,-0.020947,0.022256,-0.004138
2,0.021511,-0.032366,0.036788,0.000000,0.000000,0.124058,0.037773,0.053057,0.093899,0.015353,...,-0.058623,0.000000,-0.033571,0.000000,0.011193,0.004507,0.020120,-0.020331,-0.012140,0.000000
3,0.007138,0.000000,0.000000,-0.004327,-0.003638,-0.001787,0.000000,0.000000,0.000000,0.000000,...,0.005447,0.000000,-0.031665,0.000000,0.000000,-0.023587,0.004569,0.000000,-0.007034,0.000000
4,0.019640,0.000000,0.022095,0.031493,0.000000,0.071883,0.038417,0.078748,0.096731,0.017938,...,0.000000,-0.006971,-0.044562,-0.020881,0.000000,0.000000,0.016778,-0.009856,0.000000,0.000000


In [19]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [20]:
print(data_dict)

{'Adv sample 0': ['Bwd Header Len', 'Fwd Pkt Len Std', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Flow Byts/s', 'Pkt Len Max', 'Pkt Len Std', 'Fwd Seg Size Avg', 'Init Bwd Win Byts', 'Pkt Len Mean'], 'Adv sample 1': ['Fwd Pkt Len Std', 'Fwd Pkt Len Min', 'Flow Byts/s', 'Pkt Len Max', 'Init Bwd Win Byts', 'Pkt Len Std', 'Idle Mean', 'Idle Max', 'Bwd Seg Size Avg', 'Subflow Bwd Pkts'], 'Adv sample 2': ['Fwd Pkt Len Max', 'Fwd Pkt Len Std', 'Bwd Header Len', 'Flow Byts/s', 'Pkt Len Max', 'Fwd Pkt Len Mean', 'Fwd Seg Size Avg', 'Pkt Size Avg', 'Init Bwd Win Byts', 'Pkt Len Min'], 'Adv sample 3': ['SYN Flag Cnt', 'Bwd Seg Size Avg', 'URG Flag Cnt', 'Flow Duration', 'Fwd Seg Size Avg', 'Fwd IAT Mean', 'Fwd Act Data Pkts', 'Bwd Byts/b Avg', 'Bwd Header Len', 'Idle Mean'], 'Adv sample 4': ['Fwd Pkt Len Std', 'Bwd Header Len', 'Flow Byts/s', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Max', 'Pkt Len Std', 'Pkt Len Mean', 'Fwd Pkt Len Min', 'Fwd Seg Size Avg', 'Subflow Fwd Byts'], 'Adv sample 5': ['Fwd Pkt Le

In [21]:
len(shap_values[0])

1000

In [22]:
len(data_dict)

1000

# Extract local feature for each adv sample (77203 in total)

In [ ]:
for idx in range(0, 77203):
  # Create a DataFrame with the SHAP values and feature names
  df_shap_values = pd.DataFrame({'SHAP Values': exp[idx].values}, index=feature_list)
  df_shap_values.sort_values(by=['SHAP Values'], ascending=False, inplace=True)
  df_shap_values.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/fgsm_Result/SHAP_ADVSamples_' + str(idx) + '.csv')
  print('Extracted local importance feature of the ' + str(idx) + 'adv sample.')

# Detection phase

In [23]:
import os
import pandas as pd

In [24]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,FIN Flag Cnt,1223
1,SYN Flag Cnt,1196
2,Init Fwd Win Byts,986
3,Flow Pkts/s,884
4,ACK Flag Cnt,808
5,Pkt Size Avg,767
6,Bwd Pkts/s,752
7,Down/Up Ratio,732
8,Pkt Len Mean,667
9,Pkt Len Min,588


## Detection rate (top 24) = 1000/1000

In [25]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['FIN Flag Cnt', 'SYN Flag Cnt', 'Init Fwd Win Byts', 'Flow Pkts/s', 'ACK Flag Cnt', 'Pkt Size Avg', 'Bwd Pkts/s', 'Down/Up Ratio', 'Pkt Len Mean', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Pkt Len Max', 'Init Bwd Win Byts', 'Fwd Pkt Len Max', 'Fwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'Pkt Len Std', 'Bwd IAT Tot', 'Idle Max', 'Fwd Seg Size Avg', 'Fwd IAT Std']


In [26]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:1000


## Detection rate (top 38) = 895/1000

In [27]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['FIN Flag Cnt', 'SYN Flag Cnt', 'Init Fwd Win Byts', 'Flow Pkts/s', 'ACK Flag Cnt', 'Pkt Size Avg', 'Bwd Pkts/s', 'Down/Up Ratio', 'Pkt Len Mean', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Pkt Len Max', 'Init Bwd Win Byts', 'Fwd Pkt Len Max', 'Fwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'Pkt Len Std', 'Bwd IAT Tot', 'Idle Max', 'Fwd Seg Size Avg', 'Fwd IAT Std', 'Bwd PSH Flags', 'Flow Duration', 'Bwd IAT Std', 'Fwd Pkt Len Min', 'Flow IAT Std', 'Bwd IAT Max', 'Idle Mean', 'Fwd IAT Max', 'URG Flag Cnt', 'Idle Min', 'Flow IAT Mean', 'Fwd IAT Mean', 'Flow Byts/s', 'Bwd Pkt Len Mean']


In [28]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:895
